In [1]:
sealed abstract class BTree
case class Leaf() extends BTree
case class Node(value: Int, left: BTree, right: BTree) extends BTree

val t = Node(3, Node(1, Leaf(), Leaf()), Node(2, Leaf(), Node(4, Leaf(), Leaf())))

defined class BTree
defined class Leaf
defined class Node
t: Node = Node(
  3,
  Node(1, Leaf(), Leaf()),
  Node(2, Leaf(), Node(4, Leaf(), Leaf()))
)

In [2]:
def find(t: BTree, i: Int) : Boolean =
    t match {
        case Leaf() => false
        case Node(n,_,_) if n == i => true
        case Node(_, lt, rt) => find(lt, i) || find(rt, i)
    }

defined function find

In [3]:
def t: BTree = Node(5,Node(4,Node(2,Leaf(),Leaf()),Leaf()),
Node(7,Node(6,Leaf(),Leaf()),Leaf()))
find(t,7)
find(t,1)

defined function t
res2_1: Boolean = true
res2_2: Boolean = false

In [7]:
sealed abstract class BTList
case class BTNil() extends BTList
case class BTCons(hd: BTree, tl: BTList) extends BTList
def find(t: BTree, x: Int) : Boolean = {
    def findIter(ts: BTList) : Boolean =
        ts match {
            case BTNil() => false
            case BTCons(Leaf(), tl) => findIter(tl)
            case BTCons(Node(v, _, _), _) if v == x => true
            case BTCons(Node(_, l, r), tl) =>
                findIter(BTCons(l, BTCons(r, tl)))
        }
        findIter(BTCons(t, BTNil()))
}
def genTree(v: Int, n: Int) : BTree = {
    def genTreeIter(t: BTree, m : Int) : BTree =
        if (m == 0) t
        else genTreeIter(Node(v, t, Leaf()), m-1)
    genTreeIter(Leaf(), n)
}
find(genTree(0,100000), 1)

defined class BTList
defined class BTNil
defined class BTCons
defined function find
defined function genTree
res6_5: Boolean = false

### Typed Programming
* untyped programming: no checking at compile time
* why needs *compile-time type checking*?
  * no type errors at run time
  * no garbage collector, no free function (소유권 규칙)

### Parametric Polymorphism
* same as generic in java

In [10]:
def id[A](x: A): A = x

id("abc")
id(3)

defined function id
res9_1: String = "abc"
res9_2: Int = 3

In [12]:
// val x = id[String] _
// only monomorphic function
val x = id _ 
x("abc")

cmd12.sc:2: type mismatch;
 found   : String("abc")
 required: Nothing
val res12_1 = x("abc")
                ^Compilation Failed

: 

In [20]:
// def applyn[A](f: A => A, n: Int, x: A): A =
//     n match {
//         case 0 => x
//         case _ => f(applyn(f, n - 1, x))
//     }

// Tail Call Optimization
def applyn[A](f: A => A, n: Int, x: A): A = {
    def applynIter(i: Int, acc: A): A = {
        if (i == 0) acc
        else applynIter(i-1, f(acc))
    }
    applynIter(n, x)
}    

applyn((x:Int)=>x+1, 100, 3)
applyn((x:String)=>x+"!", 10, "gil")
applyn(id[String], 10, "hur")

defined function applyn
res19_1: Int = 103
res19_2: String = "gil!!!!!!!!!!"
res19_3: String = "hur"

In [17]:
def foo[A,B](f: A=>A, x: (A,B)) : (A,B) =
    (applyn[A](f, 10, x._1), x._2)

foo[String,Int]((x:String)=>x+"!",("abc",10))

defined function foo
res16_1: (String, Int) = ("abc!!!!!!!!!!", 10)

#### How to make `applyn` as value?

In [30]:
// val f : [A](A=>A, Int, A) => A = applyn _
type Applyn = {def apply[A](f: A=>A, n: Int, x: A): A}

val f: Applyn = new {
    def apply[A](f: A=>A, n: Int, x: A): A = applyn(f, n, x)
}

f((x:String)=>x+"!", 10, "gil")
f((x:Int)=>x+1, 100, 3)

defined type Applyn
f: Applyn = ammonite.$sess.cmd29$Helper$$anon$1@fa18dfe
res29_2: String = "gil!!!!!!!!!!"
res29_3: Int = 103

In [33]:
object applyn1 { // val applyn = new {
    def apply[A](f: A=>A, n: Int, x: A): A =
        n match {
            case 0 => x
            case _ => f(apply(f, n-1, x))
        }
}

applyn1((x:String)=>x+"!", 10, "gil")
applyn1((x:Int)=>x+1, 100, 3)

defined object applyn1
res32_1: String = "gil!!!!!!!!!!"
res32_2: Int = 103

In [38]:
// apply keyword can be removed (like def __call__ in python)
f((x:String)=>x+"!", 10, "gil")
f.apply[String]((x:String)=>x+"!", 10, "gil")

res37_0: String = "gil!!!!!!!!!!"
res37_1: String = "gil!!!!!!!!!!"

In [26]:
def foo(f: Applyn): String = {
    val a:String = f[String]((x:String)=>x+"!", 10, "gil")
    val b:Int = f[Int]((x:Int)=>x+2, 10, 5)
    a + b.toString()
}
foo(applyn)

defined function foo
res25_1: String = "gil!!!!!!!!!!25"